In [185]:
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from operator import truediv
from nxpd import draw
from collections import Counter
import itertools
from matplotlib import pyplot as plt

In [207]:
G = nx.DiGraph()

In [206]:
def insert_node(name, email, learn, teach):
    G.add_node(name, email=email, learn=learn, teach=teach)
    
    for node in G.nodes(data=True):
        learn_other = node[1]['learn']
        teach_other = node[1]['teach']
        
        common = [l for l in learn if l in teach_other]
        w = len(common)
        G.add_edge(name, node[0], weight=w, common=common)
        
        common = [l for l in teach if l in learn_other]
        w = len(common)
        G.add_edge(node[0], name, weight=w, common=common)

In [208]:
# Retrieve data from Google Sheet and parse using pandas dataframe
data = pd.read_csv("https://raw.githubusercontent.com/benelson/collaboratr/master/collaboratr.csv")
data = data.replace(np.nan,' ', regex=True)
users = data['1. What is your name?']
emails = data['2. What is your email?']
teach = data['3. What can you help others with?']
learn = data['4. What do you want to collaborate on or learn about?']

## Decision Tree

In [210]:
big_list = []
for i in range(len(teach)):
    big_list.append(teach[i].split(';'))

big_list1 = []
for i in range(len(learn)):
    big_list1.append(learn[i].split(';'))

In [211]:
all_teach = list((itertools.chain.from_iterable(big_list)))
N_teach = Counter(all_teach).values()

In [212]:
all_learn = list((itertools.chain.from_iterable(big_list1)))
N_learn = Counter(all_learn).values()

In [229]:
ratios =  map(truediv,N_learn,N_teach[:-1])

In [230]:
breakout_pct = .2
N = len(users)
subjects = Counter(all_teach).keys()
isBreakout = (np.array(ratios) > 2) & (np.array(N_learn > breakout_pct*N))

In [232]:
breakouts = np.array(subjects)[list(np.where(isBreakout)[0])]
breakouts

array(['GAIA DRI', 'stochastic gradient', 'gaussian processes',
       'databases', 'HTML5, CSS, javascript,GitHub pages', 'autodiff'], 
      dtype='|S35')

## Graphs

In [216]:
[insert_node(u, email=e, learn=l.split(';'), teach=t.split(';')) for u,e,l,t in zip(users, emails, learn, teach)]
pass

In [217]:
participants = G.nodes(data=True)

In [218]:
assign = {}
i=0
for p in participants:
    assign_p = {}
    for l in p[1]['learn']:
        if l in breakouts:
            assign_p[l] = 'breakout'
        else:
            edges = G.edges(p[0], data=True)
            for e in edges:
                if (l in e[2]['common']) and (l!=''):
                    assign_p[l] = e[1]
                else: 
                    pass
    assign[p[0]] = assign_p
    i+=1

In [219]:
assign

{'Adrian Price-Whelan': {'GAIA DRI': 'Adrian Price-Whelan'},
 'Arna Karick': {'Jupyter': 'Phil Marshall',
  'astronomy cirriculum': 'Kate Follette',
  'databases': 'Dan F-M',
  'machine learning': 'breakout',
  'sphinx': 'Dan F-M'},
 'Ben Nelson': {'autodiff': 'Dan F-M',
  'gaussian processes': 'Dan F-M',
  'machine learning': 'breakout',
  'packaging (Python)': 'Dan F-M'},
 'Brigitta Sipocz': {'autodiff': 'Dan F-M',
  'gaussian processes': 'Dan F-M',
  'machine learning': 'breakout'},
 'Casey Law': {'Jupyter': 'Phil Marshall',
  'Travis-CI': 'breakout',
  'astronomy cirriculum': 'Kate Follette',
  'machine learning': 'breakout',
  'packaging (Python)': 'Dan F-M',
  'sphinx': 'Dan F-M'},
 'Dan F-M': {'GAIA DRI': 'Adrian Price-Whelan'},
 'Jonathan Whitmore': {},
 'Kate Follette': {'Jupyter': 'Phil Marshall',
  'astronomy cirriculum': 'Kate Follette',
  'gaussian processes': 'Dan F-M'},
 'Layne Price': {'databases': 'Dan F-M', 'stochastic gradient': 'breakout'},
 'Oleg Burgue\xc3\xb1o': 

In [221]:
G1 = nx.DiGraph()

In [222]:
G1.add_nodes_from(participants)

In [223]:
G1.nodes()

['Oleg Burgue\xc3\xb1o',
 'Brigitta Sipocz',
 'Jonathan Whitmore',
 'Layne Price',
 'Ben Nelson',
 'Usman Khan',
 'Adrian Price-Whelan',
 'Zaki Ali',
 'Casey Law',
 'Dan F-M',
 'Vinayak Mathur',
 'Kate Follette',
 'Phil Marshall',
 'Arna Karick']

In [224]:
for p in G.nodes():
    mappings = assign[p]
    for m in mappings.keys():
        G1.add_edge(p, mappings[m], label=m)

In [227]:
pos=nx.spring_layout(G1)
draw(G1)

'/var/folders/pc/lq5slw9x3f92zfnsfvj0n82h0000gn/T/nx_LDuDfW.png'